In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
import torch
from infra.model_base import ModelBase
import requests
import shutil
import pathlib
import pickle

In [17]:
r = requests.get("https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz", stream=True)
download = pathlib.Path("./temp")
if not download.exists():
    download.mkdir(parents=True)
download = download / "cifar-10.tar.gz"
with download.open("wb") as fout:
    shutil.copyfileobj(r.raw, fout)

In [18]:
class TestNet(ModelBase):
    def __init__(self, checkpoint_dir):
        super(TestNet, self).__init__(checkpoint_dir)
        self.conv1 = torch.nn.Conv2d(3, 8, 3, padding="same")
        self.bn1 = torch.nn.BatchNorm2d(8)
        self.relu = torch.nn.ReLU()
        self.maxpool = torch.nn.MaxPool2d(2, 2)
        self.conv2 = torch.nn.Conv2d(8, 32, 3, padding="same")
        self.bn2 = torch.nn.BatchNorm2d(32)
        self.conv3 = torch.nn.Conv2d(32, 64, 3, padding="same")
        self.bn3 = torch.nn.BatchNorm2d(64)
        self.flatten = torch.nn.Flatten()
        self.linear = torch.nn.Linear(256, 10)

    
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.flatten(x)
        x = self.linear(x)
        return x

In [19]:
loss = torch.nn.CrossEntropyLoss()
model = TestNet("./temp/checkpoint")
optimizer = torch.optim.Adam(model.parameters(), 1e-3)

In [23]:
print(model.summary())

Model:
══════════════════════════════════════════════════
conv1: Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=same) - 224 parameters
──────────────────────────────────────────────────
bn1: BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True) - 16 parameters
──────────────────────────────────────────────────
relu: ReLU() - 0 parameters
──────────────────────────────────────────────────
maxpool: MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False) - 0 parameters
──────────────────────────────────────────────────
conv2: Conv2d(8, 32, kernel_size=(3, 3), stride=(1, 1), padding=same) - 2336 parameters
──────────────────────────────────────────────────
bn2: BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True) - 64 parameters
──────────────────────────────────────────────────
conv3: Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same) - 18496 parameters
────────────────────────────────────────────

In [21]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict
batch_1 = unpickle("./temp/cifar-10-batches-py/data_batch_1")

In [22]:
X, y = batch_1[b"data"].reshape(-1,3, 32, 32), batch_1[b"labels"]

In [ ]:
class CifarDataset(torch.utils.Dataset):
    def __init__(self,  )